# Predict Student Performance from Game Play
### Trace student learning from Jo Wilder online educational game.
## 
V. Brad Culbertson <br>
v.brad.culbertson@gmail.com <br>
https://www.linkedin.com/in/brad-culbertson <br>
https://github.com/VBradCulbertson?tab=repositories <br>
Kaggle Featured Code Competition
## 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [2]:
class Config:
    TRAIN_PATH = '../inputs/train.csv'
    TEST_PATH = '../inputs/test.csv'
    TRAIN_LABELS = '../inputs/train_labels.csv'
    SAMPLE_SUBMISSION = '../inputs/sample_submission.csv'

In [3]:
df = pd.read_csv(Config.TRAIN_PATH)
labels = pd.read_csv(Config.TRAIN_LABELS)
#df = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv')
#labels = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')

In [4]:
# Three empty columns will be dropped.
df = df.drop(columns = ['fullscreen', 'hq', 'music'])

## 
## Notebook Study
## 

### How are notebook clicks diistributed? How do they influence performance?
- Exclude closing notebook clicks since they are mandatory to progress in the game. 
- Viewing these "choice clicks" may provide insight. 
- Cluster notebook clicks:
        - 0 contains notebook clicks prior to the first checkpoint (quiz).
        - 01 contains all clicks prior to the second checkpoint.
        - 1 contains all clicks between the first and second checkpoint.
        - 12 contains all clicks prior to the third checkpoint.
        - 2 contains all clicks between the second and third checkpoint.

In [5]:
# Subset for relevant clusters.
zero = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close') & (df['level_group'] == '0-4')]
zero_one = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close') & (df['level_group'] != '13-22')]
one = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close') & (df['level_group'] == '5-12')]
one_two = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close') & (df['level_group'] != '0-4')]
two = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close') & (df['level_group'] == '13-22')]
total = df[(df['event_name'] == 'notebook_click') & (df['name'] != 'close')]

In [6]:
notebook_sets = [zero, one, two, zero_one, one_two, total]
notes = pd.DataFrame()
ctr = 0

for n in notebook_sets:
    notes['clk'] = n.groupby('session_id')['event_name'].count()
    notes['clk'].fillna(0, inplace = True)
    notes['clk'] = notes['clk'].apply(int)
    if ctr <= 2:
        notes.rename(columns = {'clk' : f'nb_clx{ctr}'}, inplace = True)
    elif ctr == 3:
        notes.rename(columns = {'clk' : 'nb_clx01'}, inplace = True)
    elif ctr == 4:
        notes.rename(columns = {'clk' : 'nb_clx12'}, inplace = True)
    else:
        notes.rename(columns = {'clk' : f'total_nb_clx'}, inplace = True)
    ctr += 1
    
notes.reset_index(inplace = True)

In [7]:
notes

,session_id,nb_clx0,nb_clx1,nb_clx2,nb_clx01,nb_clx12,total_nb_clx
0,20090312433251036,1,4,31,5,35,36
1,20090313091715820,1,3,1,4,4,5
2,20090313571836404,2,5,0,7,5,7
3,20090314035813970,6,4,2,10,6,12
4,20090314121766812,6,13,14,19,27,33
...,...,...,...,...,...,...,...
13938,22100215342220508,2,8,4,10,12,14
13939,22100215460321130,4,4,3,8,7,11
13940,22100217104993650,2,8,30,10,38,40
13941,22100219442786200,3,13,13,16,26,29


- Notebook clicks by section are validated.

### Page clicks in game notebook.
- The in-game notebook is numbered by both left and right "pages" counted as a single unit by name.
- Clicks by page provides insight into which information the individual is interacting with most frequently.
- Pages will be grouped individually, but will be included in modeling using the same clustering logic above.

In [8]:
pages = pd.DataFrame(df.groupby(['session_id', 'page'])['page'].count())
pages.columns = ['page_ct']
pages.reset_index(['page'], inplace = True)
pages = pages.pivot(columns = 'page', values = 'page_ct')
pages.rename_axis(None, axis = 1, inplace = True)
pages.rename(columns = {0 : 'P0', 1 : 'P1', 2 : 'P2',
                       3 : 'P3', 4 : 'P4', 5 : 'P5',
                       6: 'P6'}, inplace = True)
pages.fillna(value = 0, inplace = True)
pages.reset_index(inplace = True)

for p in pages.columns:
    pages[p] = pages[p].apply(int)

pages

,session_id,P0,P1,P2,P3,P4,P5,P6
0,20090312433251036,2,3,0,4,16,13,21
1,20090312455206810,0,2,0,4,8,4,14
2,20090313091715820,2,5,0,0,2,0,0
3,20090313571836404,4,0,3,4,0,0,0
4,20090314035813970,12,3,0,0,0,3,0
...,...,...,...,...,...,...,...,...
20882,22100215342220508,4,3,2,3,2,6,0
20883,22100215460321130,4,9,0,2,2,4,0
20884,22100217104993650,3,7,4,7,16,24,6
20885,22100219442786200,5,8,6,5,8,4,14


In [9]:
notes = pd.merge(notes, pages)
notes['session_id'] = notes['session_id'].apply(str)
notes

,session_id,nb_clx0,nb_clx1,nb_clx2,nb_clx01,nb_clx12,total_nb_clx,P0,P1,P2,P3,P4,P5,P6
0,20090312433251036,1,4,31,5,35,36,2,3,0,4,16,13,21
1,20090313091715820,1,3,1,4,4,5,2,5,0,0,2,0,0
2,20090313571836404,2,5,0,7,5,7,4,0,3,4,0,0,0
3,20090314035813970,6,4,2,10,6,12,12,3,0,0,0,3,0
4,20090314121766812,6,13,14,19,27,33,7,20,4,4,9,14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13938,22100215342220508,2,8,4,10,12,14,4,3,2,3,2,6,0
13939,22100215460321130,4,4,3,8,7,11,4,9,0,2,2,4,0
13940,22100217104993650,2,8,30,10,38,40,3,7,4,7,16,24,6
13941,22100219442786200,3,13,13,16,26,29,5,8,6,5,8,4,14


- Should time spent per notebook page be calculated and included for analysis?

## Responses

- Training labels were provided as a separate .csv file.
- Session_id numbers included question number in the string, they've been separated below.
- Pivoting labels by question number into a wide format using unique session id.

In [10]:
labels[['session_id', 'Q#']] = labels['session_id'].str.split('_q', expand = True)
labels

,session_id,correct,Q#
0,20090312431273200,1,1
1,20090312433251036,0,1
2,20090312455206810,1,1
3,20090313091715820,0,1
4,20090313571836404,1,1
...,...,...,...
424111,22100215342220508,1,18
424112,22100215460321130,1,18
424113,22100217104993650,1,18
424114,22100219442786200,1,18


In [11]:
labels = labels.pivot(index = 'session_id', columns = 'Q#', values = 'correct').reset_index()
column_order = ['session_id', '1', '2', '3', '4', '5', '6',
               '7', '8', '9', '10', '11', '12',
               '13', '14', '15', '16', '17', '18']
labels = labels.reindex(column_order, axis=1)
labels.head()

Q#,session_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,20090312431273200,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1
1,20090312433251036,0,1,1,1,0,1,1,0,1,0,0,1,0,1,0,1,0,1
2,20090312455206810,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1
3,20090313091715820,0,1,1,1,1,0,1,1,1,0,0,1,0,1,0,1,1,1
4,20090313571836404,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1


## Building the train dataframe.
- Append the notes and clicks features to the labels dataframe.

In [12]:
labels = pd.merge(labels, notes,
                   how = 'left',
                   on = 'session_id')
labels.fillna(0, inplace = True)

floats = ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6',
          'nb_clx0', 'nb_clx01', 'nb_clx1', 'nb_clx12', 'nb_clx2', 'total_nb_clx']

for f in floats:
    labels[f] = labels[f].apply(int)

labels.head()

,session_id,1,2,3,4,5,6,7,8,9,...,nb_clx01,nb_clx12,total_nb_clx,P0,P1,P2,P3,P4,P5,P6
0,20090312431273200,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,20090312433251036,0,1,1,1,0,1,1,0,1,...,5,35,36,2,3,0,4,16,13,21
2,20090312455206810,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,20090313091715820,0,1,1,1,1,0,1,1,1,...,4,4,5,2,5,0,0,2,0,0
4,20090313571836404,1,1,1,1,1,1,1,1,1,...,7,5,7,4,0,3,4,0,0,0


## 
## Click Latency & Hover Study
## 

- Exploring the delay between clicks may provide insight.
- Does spam-clicking influence peformance? Does a slower click pace?
- What can hover duration tell us about performance?
- Clustering logic of click latency and hover duration is consistent with notebook clicks.


In [13]:
times = df.drop(columns = ['event_name', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()
times = times[times['level_group'] == '0-4']
times = times.drop(columns = 'level_group')

temp0 = pd.DataFrame()
temp0['avg_clk_diff0'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
temp0['avg_hover_dur0'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
temp0['avg_hover_dur0'].fillna(0, inplace = True)
temp0 = temp0.reset_index()
temp0

,session_id,avg_clk_diff0,avg_hover_dur0
0,20090312431273200,1180.969697,2389.500000
1,20090312433251036,1681.669065,1378.750000
2,20090312455206810,2437.758389,3145.666667
3,20090313091715820,1095.414773,1917.142857
4,20090313571836404,1748.669643,1580.714286
...,...,...,...
23557,22100215342220508,3697.566667,2736.727273
23558,22100215460321130,2387.912752,1439.250000
23559,22100217104993650,2079.487310,2765.666667
23560,22100219442786200,1148.065868,3337.500000


In [14]:
temp0.isna().sum()

session_id        0
avg_clk_diff0     0
avg_hover_dur0    0
dtype: int64

In [15]:
times = df.drop(columns = ['event_name', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()
times = times[times['level_group'] != '13-22']
times = times.drop(columns = 'level_group')

temp01 = pd.DataFrame()
temp01['avg_clk_diff01'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
temp01['avg_hover_dur01'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
temp01['avg_hover_dur01'].fillna(0)
temp01 = temp01.reset_index()
temp01

,session_id,avg_clk_diff01,avg_hover_dur01
0,20090312431273200,1082.939262,1268.315789
1,20090312433251036,1531.102996,887.485714
2,20090312455206810,1606.243655,1027.448276
3,20090313091715820,1492.633466,1566.282609
4,20090313571836404,1490.443503,1432.040000
...,...,...,...
23557,22100215342220508,3494.289532,3949.755556
23558,22100215460321130,2003.357815,1377.107692
23559,22100217104993650,1671.174873,2390.309524
23560,22100219442786200,1262.184270,1682.823529


In [16]:
times = df.drop(columns = ['event_name', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()
times = times[times['level_group'] == '5-12']
times = times.drop(columns = 'level_group')

temp1 = pd.DataFrame()
temp1['avg_clk_diff1'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
temp1['avg_hover_dur1'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
temp1['avg_hover_dur1'].fillna(0)
temp1 = temp1.reset_index()
temp1

,session_id,avg_clk_diff1,avg_hover_dur1
0,20090312431273200,938.344595,969.333333
1,20090312433251036,1394.243038,824.096774
2,20090312455206810,1030.383673,783.038462
3,20090313091715820,1584.184049,1503.307692
4,20090313571836404,1267.925620,1374.222222
...,...,...,...
23557,22100215342220508,3235.096990,4342.205882
23558,22100215460321130,1818.274869,1368.385965
23559,22100217104993650,1385.198980,2327.750000
23560,22100219442786200,1165.960432,1328.250000


In [17]:
times = df.drop(columns = ['event_name', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()
times = times[times['level_group'] != '0-4']
times = times.drop(columns = 'level_group')

temp12 = pd.DataFrame()
temp12['avg_clk_diff12'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
temp12['avg_hover_dur12'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
temp12['avg_hover_dur12'].fillna(0)
temp12 = temp12.reset_index()
pd.set_option('display.float_format', lambda x: '%.5f' % x) # A few outliers forced auto-formatting to scientific notation.
temp12

,session_id,avg_clk_diff12,avg_hover_dur12
0,20090312431273200,1468.14804,936.45614
1,20090312433251036,2097.19326,740.86306
2,20090312455206810,1327.80624,916.50980
3,20090313091715820,1699.52264,1273.88312
4,20090313571836404,1504.56940,1409.46154
...,...,...,...
23557,22100215342220508,79032.30705,765549.60563
23558,22100215460321130,2075.71651,1079.62500
23559,22100217104993650,1659.24251,1691.34247
23560,22100219442786200,1399.91726,1227.75000


In [18]:
times = df.drop(columns = ['event_name', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()
times = times[times['level_group'] == '13-22']
times = times.drop(columns = 'level_group')

temp2 = pd.DataFrame()
temp2['avg_clk_diff2'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
temp2['avg_hover_dur2'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
temp2['avg_hover_dur2'].fillna(0)
temp2 = temp2.reset_index()
pd.set_option('display.float_format', lambda x: '%.5f' % x)
temp2

,session_id,avg_clk_diff2,avg_hover_dur2
0,20090312431273200,1037.96905,899.92593
1,20090312433251036,2034.58057,720.38492
2,20090312455206810,1212.59066,1055.32000
3,20090313091715820,1313.65377,1038.42105
4,20090313571836404,1247.78571,1439.66667
...,...,...,...
23557,22100215342220508,131663.34670,1465037.48649
23558,22100215460321130,1584.80176,780.36364
23559,22100217104993650,1498.92951,1072.13514
23560,22100219442786200,1270.85816,1110.50000


In [19]:
times = df.drop(columns = ['event_name', 'level_group', 'index', 'name', 'page', 'room_coor_x', 'room_coor_y',
                           'screen_coor_x', 'screen_coor_y', 'text', 'fqid', 'room_fqid',
                           'text_fqid']).sort_values(['session_id', 'elapsed_time']).reset_index()

tempt = pd.DataFrame()
tempt['avg_clk_diff_tot'] = (times.groupby('session_id')['elapsed_time'].max() - times.groupby('session_id')['elapsed_time'].min()) / times.groupby('session_id')['elapsed_time'].count()
tempt['avg_hover_dur_tot'] = times.groupby('session_id')['hover_duration'].sum() / times.groupby('session_id')['hover_duration'].count()
tempt['avg_hover_dur_tot'].fillna(0)
tempt = tempt.reset_index()
pd.set_option('display.float_format', lambda x: '%.5f' % x)
tempt

,session_id,avg_clk_diff_tot,avg_hover_dur_tot
0,20090312431273200,1444.58456,1115.29231
1,20090312433251036,2083.74331,756.71118
2,20090312455206810,1568.66755,1040.35185
3,20090313091715820,1632.79758,1327.48810
4,20090313571836404,1574.63271,1435.52174
...,...,...,...
23557,22100215342220508,66142.03895,663221.04878
23558,22100215460321130,2136.54924,1103.60000
23559,22100217104993650,1754.69975,1772.93671
23560,22100219442786200,1404.23618,1446.00000


In [20]:
temps = [temp0, temp01, temp1, temp12, temp2, tempt]

for t in temps:
    
    labels['session_id'] = labels['session_id'].apply(str)
    t['session_id'] = t['session_id'].apply(str)
    labels = pd.merge(labels, t,
                how = 'left',
                on = 'session_id')

## Cluster score for quizzes.
- Overall performance on each quiz will be available for future quiz predictions.
- The last quiz score won't be available for modeling, but it is included for EDA purposes.

In [21]:
labels['score0'] = (labels['1'] + labels['2'] + labels['3']) / 3
labels['score1'] = (labels['4'] + labels['5'] + labels['6'] + labels['7'] +
                    labels['8'] + labels['9'] + labels['10'] + labels['11'] +
                    labels['12'] + labels['13']) / 10
labels['score2'] = (labels['14'] + labels['15'] + labels['16'] + labels['17'] +
                    labels['18']) / 5

In [22]:
pd.set_option('display.max_columns', None)
labels.head()

,session_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,nb_clx0,nb_clx1,nb_clx2,nb_clx01,nb_clx12,total_nb_clx,P0,P1,P2,P3,P4,P5,P6,avg_clk_diff0,avg_hover_dur0,avg_clk_diff01,avg_hover_dur01,avg_clk_diff1,avg_hover_dur1,avg_clk_diff12,avg_hover_dur12,avg_clk_diff2,avg_hover_dur2,avg_clk_diff_tot,avg_hover_dur_tot,score0,score1,score2
0,20090312431273200,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1180.96970,2389.50000,1082.93926,1268.31579,938.34459,969.33333,1468.14804,936.45614,1037.96905,899.92593,1444.58456,1115.29231,1.00000,0.90000,0.80000
1,20090312433251036,0,1,1,1,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,4,31,5,35,36,2,3,0,4,16,13,21,1681.66906,1378.75000,1531.10300,887.48571,1394.24304,824.09677,2097.19326,740.86306,2034.58057,720.38492,2083.74331,756.71118,0.66667,0.50000,0.60000
2,20090312455206810,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2437.75839,3145.66667,1606.24365,1027.44828,1030.38367,783.03846,1327.80624,916.50980,1212.59066,1055.32000,1568.66755,1040.35185,1.00000,1.00000,0.80000
3,20090313091715820,0,1,1,1,1,0,1,1,1,0,0,1,0,1,0,1,1,1,1,3,1,4,4,5,2,5,0,0,2,0,0,1095.41477,1917.14286,1492.63347,1566.28261,1584.18405,1503.30769,1699.52264,1273.88312,1313.65377,1038.42105,1632.79758,1327.48810,0.66667,0.60000,0.80000
4,20090313571836404,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,2,5,0,7,5,7,4,0,3,4,0,0,0,1748.66964,1580.71429,1490.44350,1432.04000,1267.92562,1374.22222,1504.56940,1409.46154,1247.78571,1439.66667,1574.63271,1435.52174,1.00000,0.90000,0.80000


## 
## Room Study
## 

- How much time is the player spending in each room?
- Does time spent per room predict performance overall, or related to that room?

In [23]:
df['room_fqid'].unique()

array(['tunic.historicalsociety.closet',
       'tunic.historicalsociety.basement',
       'tunic.historicalsociety.entry',
       'tunic.historicalsociety.collection',
       'tunic.historicalsociety.stacks', 'tunic.kohlcenter.halloffame',
       'tunic.capitol_0.hall', 'tunic.historicalsociety.closet_dirty',
       'tunic.historicalsociety.frontdesk',
       'tunic.humanecology.frontdesk', 'tunic.drycleaner.frontdesk',
       'tunic.library.frontdesk', 'tunic.library.microfiche',
       'tunic.capitol_1.hall', 'tunic.historicalsociety.cage',
       'tunic.historicalsociety.collection_flag', 'tunic.wildlife.center',
       'tunic.flaghouse.entry', 'tunic.capitol_2.hall'], dtype=object)

In [24]:
df = df.sort_values(['session_id', 'elapsed_time', 'room_fqid'])
df['same_room'] = df.room_fqid.eq(df.room_fqid.shift())

## Find time spent in each room.

In [25]:
# Calculate time in room from entry current room to entry next room.
def calculate_time_in_room(session_data):
    session_data = session_data.sort_values('elapsed_time')
    session_data['same_room'] = session_data['room_fqid'].shift(1) == session_data['room_fqid']
    room_changes = session_data[session_data['same_room'] == False]
    
    room_changes['out'] = room_changes['elapsed_time'].shift(-1)
    room_changes.loc[room_changes['out'].isna(), 'out'] = session_data['elapsed_time'].max()
    
    room_changes['time_in_room'] = room_changes['out'] - room_changes['elapsed_time']
    
    return room_changes

room_time = pd.DataFrame(columns=['session_id', 'elapsed_time',
                                  'room_fqid', 'same_room',
                                  'out', 'time_in_room'])

grouped_sessions = df.groupby('session_id')

for session_id, session_data in grouped_sessions:
    room_time_for_session = calculate_time_in_room(session_data)
    room_time = pd.concat([room_time, room_time_for_session])

room_time = room_time.reset_index(drop=True)

#### Group data for room analysis by level group for predictive accuracy.

In [26]:
rooms = room_time.groupby(['session_id', 'room_fqid'])['time_in_room'].sum().reset_index()
rooms = rooms.pivot(index = 'session_id', columns = 'room_fqid', values = 'time_in_room')
rooms.rename_axis(None, axis = 1, inplace = True)
rooms.head()

,tunic.capitol_0.hall,tunic.capitol_1.hall,tunic.capitol_2.hall,tunic.drycleaner.frontdesk,tunic.flaghouse.entry,tunic.historicalsociety.basement,tunic.historicalsociety.cage,tunic.historicalsociety.closet,tunic.historicalsociety.closet_dirty,tunic.historicalsociety.collection,tunic.historicalsociety.collection_flag,tunic.historicalsociety.entry,tunic.historicalsociety.frontdesk,tunic.historicalsociety.stacks,tunic.humanecology.frontdesk,tunic.kohlcenter.halloffame,tunic.library.frontdesk,tunic.library.microfiche,tunic.wildlife.center
session_id,,,,,,,,,,,,,,,,,,,
20090312431273200,64445.00000,356600.00000,1971.00000,31106.00000,27499.00000,72645.00000,62238.00000,50985.00000,40147.00000,25565.00000,11933.00000,137984.00000,67461.00000,55795.00000,34880.00000,26800.00000,88661.00000,30820.00000,85144.00000
20090312433251036,78090.00000,398746.00000,184319.00000,169750.00000,103829.00000,109127.00000,142747.00000,25243.00000,104132.00000,89645.00000,54224.00000,664530.00000,169825.00000,223303.00000,124740.00000,196895.00000,646374.00000,143466.00000,186349.00000
20090312455206810,52028.00000,146303.00000,1959.00000,27453.00000,30402.00000,61542.00000,49422.00000,228634.00000,24250.00000,27002.00000,8449.00000,167105.00000,59187.00000,64219.00000,24519.00000,34123.00000,83301.00000,32496.00000,66656.00000
20090313091715820,105780.00000,243218.00000,19604.00000,48253.00000,46439.00000,97646.00000,84541.00000,30707.00000,85282.00000,36390.00000,19631.00000,188448.00000,105949.00000,89209.00000,66153.00000,45789.00000,113015.00000,61479.00000,133835.00000
20090313571836404,61366.00000,164531.00000,18971.00000,33517.00000,40750.00000,62750.00000,88699.00000,41482.00000,67481.00000,39832.00000,12266.00000,147855.00000,71695.00000,57985.00000,39016.00000,26317.00000,80669.00000,35362.00000,84132.00000


In [27]:
capitol = ['tunic.capitol_0.hall', 'tunic.capitol_1.hall', 'tunic.capitol_2.hall']

others = ['tunic.drycleaner.frontdesk', 'tunic.flaghouse.entry',
          'tunic.humanecology.frontdesk', 'tunic.kohlcenter.halloffame',
          'tunic.wildlife.center']
library = ['tunic.library.frontdesk', 'tunic.library.microfiche']
history = ['tunic.historicalsociety.basement', 'tunic.historicalsociety.cage',
           'tunic.historicalsociety.closet', 'tunic.historicalsociety.closet_dirty',
           'tunic.historicalsociety.collection', 'tunic.historicalsociety.collection_flag',
           'tunic.historicalsociety.entry', 'tunic.historicalsociety.frontdesk',
           'tunic.historicalsociety.stacks'] 

#### Visualizing distributions using IQR thresholds.

#Will need: from scipy.stats import norm, probplot, zscore, iqr
from scipy.stats import norm, probplot, zscore, iqr
from scipy import stats
# Calculating outliers using standard IQR multiplied by threshold:
def IQR_Outliers_Trim_Test(df,col,threshold,level='Data'):
    #Will need: from scipy.stats import norm, probplot, zscore, iqr
    untrimmed = df[col]
    q1, q3 = np.percentile(untrimmed, [25, 75])
    iqr = q3 - q1
    lower_threshold = q1 - (threshold * iqr)
    upper_threshold = q3 + (threshold * iqr)
    trimmed = df.loc[(df[col] < upper_threshold) &
             (df[col] > lower_threshold),
             [col]]
    outliers = df[(untrimmed < lower_threshold) | (untrimmed > upper_threshold)]
    
    dict = {'Untrimmed' : [threshold, df.describe()[col]['count'], 0, df.describe()[col]['mean'],
                          df.describe()[col]['std'], df.describe()[col]['min'],
                          df.describe()[col]['25%'], df.describe()[col]['50%'],
                          df.describe()[col]['75%'], df.describe()[col]['max'],
                          stats.describe(df[col]).skewness, stats.describe(df[col]).kurtosis],
           'Trimmed' : [threshold, trimmed.describe()[col]['count'],
                        (1 - (trimmed.describe()[col]['count'])/(df.describe()[col]['count'])) * 100,
                        trimmed.describe()[col]['mean'],
                        trimmed.describe()[col]['std'], trimmed.describe()[col]['min'],
                        trimmed.describe()[col]['25%'], trimmed.describe()[col]['50%'],
                        trimmed.describe()[col]['75%'], trimmed.describe()[col]['max'],
                        stats.describe(trimmed[col]).skewness, stats.describe(trimmed[col]).kurtosis]}

    index_labels = ['Threshold', 'Count', '% Trimmed', 'Mean', 'StdDev', 'Min', '25%',
                    '50%', '75%', 'Max', 'Skewness', 'Kurtosis']
    compare = pd.DataFrame(dict, index = index_labels)
    ###
    fig = make_subplots(rows=1, cols=2)
    trace1 = go.Box(y = df[col], name = f'Untrimmed', boxmean = True)
    fig.append_trace(trace1, row = 1, col = 1)
    trace2 = go.Box(y = trimmed[col], name = f'Trimmed', boxmean = True)
    fig.append_trace(trace2, row = 1, col = 2)

    fig.update_layout(height = 500, width = 900,
                     title_text = f'Untrimmed Vs. Trimmed {col}')
    fig.update_xaxes(tickangle=45)
    fig.show()
    
    return compare    

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.iqr.html <- combine iqr calc lines

IQR_Outliers_Trim_Test(rooms, 'tunic.capitol_0.hall', threshold = 1.5)

In [28]:
# Calculate time in room from entry current room to entry next room.
def calculate_time_in_room(session_data):
    session_data = session_data.sort_values('elapsed_time')
    session_data['same_room'] = session_data['room_fqid'].shift(1) == session_data['room_fqid']
    room_changes = session_data[session_data['same_room'] == False]
    
    room_changes['out'] = room_changes['elapsed_time'].shift(-1)
    room_changes.loc[room_changes['out'].isna(), 'out'] = session_data['elapsed_time'].max()
    
    room_changes['time_in_room'] = room_changes['out'] - room_changes['elapsed_time']
    
    return room_changes

In [29]:
def room_split_by_level_group(quiz_num):
    room_time = pd.DataFrame(columns=['session_id', 'elapsed_time',
                                      'room_fqid', 'same_room',
                                      'out', 'time_in_room'])

    if quiz_num == 0:
        grouped_sessions = df[df['level_group'] == '0-4'].groupby('session_id')
    elif quiz_num == 1:
        grouped_sessions = df[df['level_group'] == '5-12'].groupby('session_id')
    elif quiz_num == 2:
        grouped_sessions = df[df['level_group'] == '13-22'].groupby('session_id')

    for session_id, session_data in grouped_sessions:
        room_time_for_session = calculate_time_in_room(session_data)
        room_time = pd.concat([room_time, room_time_for_session])

    room_time = room_time.reset_index(drop=True)
    
    rooms = room_time.groupby(['session_id', 'room_fqid'])['time_in_room'].sum().reset_index()
    rooms = rooms.pivot(index = 'session_id', columns = 'room_fqid', values = 'time_in_room')
    rooms.rename_axis(None, axis = 1, inplace = True)
    return rooms

### Predictive Features for Questions 1-3

In [30]:
room_times0 = room_split_by_level_group(0).reset_index()
room_times0 = room_times0.fillna(0)
room_times0['session_id'] = room_times0['session_id'].apply(str)

labels0 = labels[['session_id', '1', '2', '3', 'nb_clx0', 'P0', 'avg_clk_diff0', 'avg_hover_dur0']]
labels0 = pd.merge(labels0, room_times0)
labels0

,session_id,1,2,3,nb_clx0,P0,avg_clk_diff0,avg_hover_dur0,tunic.capitol_0.hall,tunic.historicalsociety.basement,tunic.historicalsociety.closet,tunic.historicalsociety.collection,tunic.historicalsociety.entry,tunic.historicalsociety.stacks,tunic.kohlcenter.halloffame
0,20090312431273200,1,1,1,0,0,1180.96970,2389.50000,30837.00000,10610.00000,50985.00000,25565.00000,44796.00000,5267.00000,26800.00000
1,20090312433251036,0,1,1,1,2,1681.66906,1378.75000,37409.00000,5369.00000,25243.00000,89645.00000,45899.00000,0.00000,30187.00000
2,20090312455206810,1,1,1,0,0,2437.75839,3145.66667,28744.00000,10920.00000,228634.00000,27002.00000,33803.00000,0.00000,34123.00000
3,20090313091715820,0,1,1,1,2,1095.41477,1917.14286,50213.00000,10107.00000,30707.00000,27549.00000,28428.00000,0.00000,45789.00000
4,20090313571836404,1,1,1,2,4,1748.66964,1580.71429,31920.00000,3467.00000,41482.00000,39832.00000,52833.00000,0.00000,26317.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23557,22100215342220508,1,1,1,2,4,3697.56667,2736.72727,79182.00000,12774.00000,128005.00000,100841.00000,118462.00000,44559.00000,70812.00000
23558,22100215460321130,0,1,1,4,4,2387.91275,1439.25000,69407.00000,8479.00000,74545.00000,83777.00000,67684.00000,8278.00000,43629.00000
23559,22100217104993650,1,1,1,2,3,2079.48731,2765.66667,40411.00000,10247.00000,81101.00000,50914.00000,98976.00000,86624.00000,41386.00000
23560,22100219442786200,0,1,1,3,5,1148.06587,3337.50000,38575.00000,3397.00000,62890.00000,32018.00000,28466.00000,0.00000,26381.00000


### Predictive Features for Questions 4-13

In [31]:
room_times1 = room_split_by_level_group(1).reset_index()
room_times1 = room_times1.fillna(0)
room_times1['session_id'] = room_times1['session_id'].apply(str)

labels1 = labels[['session_id',
                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
                  'nb_clx0', 'nb_clx01', 'nb_clx1',
                  'P0', 'P1', 'P2', 'P3',
                  'avg_clk_diff0', 'avg_clk_diff01', 'avg_clk_diff1',
                  'avg_hover_dur0', 'avg_hover_dur01', 'avg_hover_dur1', 'score0']]
labels1 = pd.merge(labels1, room_times1)
labels1

,session_id,1,2,3,4,5,6,7,8,9,10,11,12,13,nb_clx0,nb_clx01,nb_clx1,P0,P1,P2,P3,avg_clk_diff0,avg_clk_diff01,avg_clk_diff1,avg_hover_dur0,avg_hover_dur01,avg_hover_dur1,score0,tunic.capitol_0.hall,tunic.capitol_1.hall,tunic.drycleaner.frontdesk,tunic.historicalsociety.basement,tunic.historicalsociety.closet_dirty,tunic.historicalsociety.collection,tunic.historicalsociety.entry,tunic.historicalsociety.frontdesk,tunic.historicalsociety.stacks,tunic.humanecology.frontdesk,tunic.kohlcenter.halloffame,tunic.library.frontdesk,tunic.library.microfiche
0,20090312431273200,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1180.96970,1082.93926,938.34459,2389.50000,1268.31579,969.33333,1.00000,6983.00000,13439.00000,31106.00000,6334.00000,40147.00000,0.00000,20406.00000,48980.00000,30494.00000,34880.00000,0.00000,30916.00000,14065.00000
1,20090312433251036,0,1,1,1,0,1,1,0,1,0,0,1,0,1,5,4,2,3,0,4,1681.66906,1531.10300,1394.24304,1378.75000,887.48571,824.09677,0.66667,7550.00000,35366.00000,56167.00000,17945.00000,57581.00000,0.00000,52354.00000,89101.00000,80722.00000,33097.00000,0.00000,57243.00000,63600.00000
2,20090312455206810,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,2437.75839,1606.24365,1030.38367,3145.66667,1027.44828,783.03846,1.00000,6094.00000,18681.00000,27453.00000,11170.00000,24250.00000,0.00000,32550.00000,23856.00000,24181.00000,24519.00000,0.00000,44574.00000,15116.00000
3,20090313091715820,0,1,1,1,1,0,1,1,1,0,0,1,0,1,4,3,2,5,0,0,1095.41477,1492.63347,1584.18405,1917.14286,1566.28261,1503.30769,0.66667,15502.00000,7651.00000,48253.00000,14003.00000,74243.00000,0.00000,42535.00000,71559.00000,53435.00000,66153.00000,0.00000,77273.00000,45837.00000
4,20090313571836404,1,1,1,1,1,1,1,1,1,1,1,0,1,2,7,5,4,0,3,4,1748.66964,1490.44350,1267.92562,1580.71429,1432.04000,1374.22222,1.00000,4518.00000,1972.00000,33517.00000,8161.00000,50821.00000,0.00000,17453.00000,52826.00000,28223.00000,39016.00000,0.00000,50296.00000,20035.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23557,22100215342220508,1,1,1,1,1,1,1,0,1,1,1,1,0,2,10,8,4,3,2,3,3697.56667,3494.28953,3235.09699,2736.72727,3949.75556,4342.20588,1.00000,82663.00000,1968.00000,82156.00000,9988.00000,93730.00000,10035.00000,61051.00000,216320.00000,160749.00000,98474.00000,0.00000,98709.00000,51451.00000
23558,22100215460321130,0,1,1,1,0,1,1,0,1,0,1,1,0,4,8,4,4,9,0,2,2387.91275,2003.35782,1818.27487,1439.25000,1377.10769,1368.38596,0.66667,30385.00000,20582.00000,69650.00000,8381.00000,89055.00000,20923.00000,40540.00000,102245.00000,46437.00000,105147.00000,0.00000,89070.00000,72166.00000
23559,22100217104993650,1,1,1,1,1,1,1,1,1,0,1,1,1,2,10,8,3,7,4,7,2079.48731,1671.17487,1385.19898,2765.66667,2390.30952,2327.75000,1.00000,14292.00000,1977.00000,112788.00000,6515.00000,70024.00000,7666.00000,33358.00000,59791.00000,44101.00000,81242.00000,0.00000,49249.00000,61995.00000
23560,22100219442786200,0,1,1,1,1,1,1,0,1,0,1,1,0,3,16,13,5,8,6,5,1148.06587,1262.18427,1165.96043,3337.50000,1682.82353,1328.25000,0.66667,10254.00000,1972.00000,27997.00000,6901.00000,35221.00000,0.00000,20082.00000,64298.00000,42628.00000,33621.00000,0.00000,52754.00000,28409.00000


### Predictive Features for Questions 14-18

In [ ]:
room_times2 = room_split_by_level_group(2).reset_index()
room_times2 = room_times1.fillna(0)
room_times2['session_id'] = room_times2['session_id'].apply(str)

labels2 = labels[['session_id',
                  '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '10', '11', '12', '13', '14', '15', '16', '17', '18',
                  'nb_clx0', 'nb_clx01', 'nb_clx1', 'nb_clx12', 'nb_clx2', 
                  'P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6',
                  'avg_clk_diff0', 'avg_clk_diff01', 'avg_clk_diff1',
                  'avg_clk_diff12', 'avg_clk_diff2', 'avg_clk_diff_tot', 
                  'avg_hover_dur0', 'avg_hover_dur01', 'avg_hover_dur1',
                  'avg_hover_dur12', 'avg_hover_dur2', 'avg_hover_dur_tot',
                 'score0', 'score1']]
labels2 = pd.merge(labels2, room_times2)
labels2

In [ ]:
labels1 = labels1.fillna(0)

In [ ]:
labels2 = labels2.fillna(0)

In [ ]:
labels0.to_csv('c://Users/Victor/Documents/Data/Gameplay Predictions/train_labels0.csv')
labels1.to_csv('c://Users/Victor/Documents/Data/Gameplay Predictions/train_labels1.csv')
labels2.to_csv('c://Users/Victor/Documents/Data/Gameplay Predictions/train_labels2.csv')